# Select:


* Pairs with high agreement/few contradictions
* Pairs with low agreement/few contradictions (if possible)
* Pairs with high agrement/many contradictions
* Pairs with low agreement/many contradictions

Try to spread over property types


# Annotate:

* New run
* Define new experiment group somehow

# Ideas:

* Low agreement, low contradiction rate: knowledge issue (some people know, some don't)
* Low agreement, high contradiction rate: ambiguity



# Implementation plan:

* sort according to agreement
* sort according to contradiction rate
* get top pairs (top 5 or so)
* get intersections:
    * most problematic: low agreement - high contradiction
    * problematic: low agreement - low contradiction
    * interesting: high agreement - high contradiction (multiple interpretations?)
    * clear: high agreement - low contradiction

In [11]:
from load_data import load_experiment_data
from calculate_iaa import get_agreement
from utils_analysis import load_contradiction_pairs
from utils_analysis import collect_contradictions
from utils_analysis import sort_by_key
from utils_analysis import get_annotation_ids
from utils_analysis import load_analysis

from collections import Counter, defaultdict
import pandas as pd
from scipy.stats import spearmanr


def get_agreement_by_property(data_dict_list):

    agreement_prop_dict = dict()
    data_by_pair = sort_by_key(data_dict_list, ['property', 'concept'])
    for pair, dl_prop in data_by_pair.items():
        agreement_prop_dict[pair] = get_agreement(dl_prop, v=False, disable_kappa=True)
    return agreement_prop_dict


def get_agreement_contradiction_data(run, group, batch, n_q):
    data_dict_list = load_experiment_data(run, group, n_q, batch, remove_not_val = True)
    print(run, group, batch, n_q)
    #data_by_pair = sort_by_key(data_dict_list, ['property', 'concept'])
    analysis_type = 'pairs'
    df = load_analysis(analysis_type, run, group, batch)
    pair_dicts = df.to_dict('records') 
    pair_dicts_by_pair = sort_by_key(pair_dicts, ['pair'])
    ag_pair_dict = get_agreement_by_property(data_dict_list)

    agreement_dict = Counter()
    for pair, ag in ag_pair_dict.items():
        agreement_dict[pair] =  ag['Krippendorff']

    ag_cont_dicts = []
    for pair, ag in agreement_dict.most_common():
        if 'test' not in pair and 'check' not in pair: 
            d = pair_dicts_by_pair[pair]
            if len(d) == 1:
                new_dict = dict()
                d = d[0]
                #print(d.keys())
                cont = d['contradiction_poss_contradiction_ratio']
                new_dict['pair'] = pair
                new_dict['agreement'] = ag
                new_dict['contradiction_rate'] = cont
                ag_cont_dicts.append(new_dict)

            else:
                print('unexpected length:', len(d), 'for pair', pair)
        else:
            print('test pair:', pair)
        #d = data_by_pair[pair]
    df_ag_cont = pd.DataFrame(ag_cont_dicts)
    return df_ag_cont



In [24]:
run = "*"
group = 'experiment*'
batch = '*'
n_q = '*'

df_cont_ag = get_agreement_contradiction_data(run, group, batch, n_q)


Discarded 655.0 annotations.
* experiment* * *
test pair: _check2-_check2
test pair: _test4-_test
test pair: _test3-_test
test pair: _check3-_check3
test pair: _test1-_test1
test pair: _test2-_test2
test pair: _check4-_check4
test pair: _check1-_check1
test pair: _test4-_test4
test pair: _test3-_test3


In [26]:
# High agreement

df_ag_sorted = df_cont_ag.sort_values('agreement', axis = 0, ascending=False, inplace=False)
top_ag = df_ag_sorted[:30]
print(f'Total number of pairs: {len(df_ag_sorted)}')
top_ag

Total number of pairs: 1538


,agreement,contradiction_rate,pair
0,1.000000,0.000000,yellow-citrus
1,0.960384,0.000000,sweet-cookie
2,0.857143,0.000000,wheels-motorcar
3,0.850160,0.033333,hot-cooker
4,0.834237,0.000000,fly-lorry
5,0.828163,0.000000,sweet-doughnut
6,0.827780,0.000000,sweet-raspberry
7,0.819202,0.000000,made_of_wood-floorboard
8,0.815993,0.000000,fly-icteridae
9,0.812376,0.000000,wheels-auto


In [17]:
# Low agreement

df_ag_sorted = df_cont_ag.sort_values('agreement', axis = 0, ascending=False, inplace=False)
bottom_ag = df_ag_sorted[-30:-1]
bottom_ag

,agreement,contradiction_rate,pair
1508,-0.029106,0.013333,fly-lynx
1509,-0.030691,0.000000,yellow-skunk
1510,-0.032245,0.000000,wheels-motor
1511,-0.035556,0.200000,square-recliner
1512,-0.036657,0.000000,round-pen
1513,-0.037281,0.000000,square-server
1514,-0.038033,0.081250,round-corkscrew
1515,-0.045636,0.000000,red-carrot
1516,-0.047619,0.000000,yellow-night
1517,-0.049601,0.186667,lay_eggs-hobby


In [27]:
# middle

df_ag_sorted = df_cont_ag.sort_values('agreement', axis = 0, ascending=False, inplace=False)
bottom_ag = df_ag_sorted[-725:-725+30]
bottom_ag

,agreement,contradiction_rate,pair
813,0.227415,0.084416,red-onion
814,0.227006,0.112500,red-frog
815,0.226562,0.000000,black-aubergine
816,0.226080,0.000000,yellow-bee
817,0.225874,0.075000,round-dessert
818,0.225649,0.000000,black-dolphin
819,0.225600,0.183333,dangerous-killer
820,0.225555,0.115000,roll-tile
821,0.225490,0.016667,made_of_wood-kickstand
822,0.225490,0.058333,dangerous-iodine


In [19]:
def pairs_to_file(name, pairs):
    path = f'../analyses/{name}.txt'
    with open(path, 'w') as outfile:
        for p in pairs:
            outfile.write(p+'\n')
    print(f'Written pairs to {path}')
    

In [28]:
# Selection for a trial round:

pair_dict = dict()
all_pairs = []

pair_dict['expert_inspection1'] = ['fly-arrow', 'roll-pin', 'yellow-buttercup',
                                   'hot-vineigrette', 'roll-tire']
pair_dict['expert_inspection2'] = ['roll-shovel', 'dangerous-freebooter',
                                   'yellow-pineapple', 'red-carrot', 'red-wine']
pair_dict['expert_inspection3'] = ['square-recliner', 'yellow-leopard', 
                                   'round-pen', 'roll-washer', 'lay_eggs-plaice']
pair_dict['expert_inspection4'] = ['dangerous-iodine','black-cockroach', 
                                   'wheels-cruiser','yellow-saffron', 'sweet-tangerine']


all_pairs = []
for name, pairs in pair_dict.items():
    if any([p in all_pairs for p in pairs]):
        print(f'Attention! Duplicate pair in {name}')
    else:
        all_pairs.extend(pairs)
        pairs_to_file(name, pairs)
        

Written pairs to ../analyses/expert_inspection1.txt
Written pairs to ../analyses/expert_inspection2.txt
Written pairs to ../analyses/expert_inspection3.txt
Written pairs to ../analyses/expert_inspection4.txt
